In [10]:
import org.apache.spark.sql.types.{StructType, StructField, StringType}
import org.apache.spark.sql.functions.{col}
import org.apache.spark.sql._
import spark.implicits._

import org.apache.spark.sql.types.{StructType, StructField, StringType}
import org.apache.spark.sql.functions.col
import org.apache.spark.sql._
import spark.implicits._


In [11]:
val graph = Seq(("1","2"),("1","3"),("2","3"),("2","1"))
val graphRDD = sc.parallelize(graph)

graph: Seq[(String, String)] = List((1,2), (1,3), (2,3), (2,1))
graphRDD: org.apache.spark.rdd.RDD[(String, String)] = ParallelCollectionRDD[4] at parallelize at <console>:42


In [12]:
def seqOp = (accumulator: List[String], element: String) => 
    accumulator:+element

def combOp = (accumulator1: List[String], accumulator2: List[String]) => 
    accumulator1 ++ accumulator2

val zeroVal = ()
val following = graphRDD.aggregateByKey(List[String]())(seqOp, combOp)

seqOp: (List[String], String) => List[String]
combOp: (List[String], List[String]) => List[String]
zeroVal: Unit = ()
following: org.apache.spark.rdd.RDD[(String, List[String])] = ShuffledRDD[5] at aggregateByKey at <console>:49


In [13]:
following.collect()

res2: Array[(String, List[String])] = Array((1,List(2, 3)), (2,List(3, 1)))


In [14]:
def seqOp = (accumulator: List[String], element: String) => 
    accumulator:+element

def combOp = (accumulator1: List[String], accumulator2: List[String]) => 
    accumulator1 ++ accumulator2

val zeroVal = ()
val followers = graphRDD.map{case(k,v) => (v,k)}.aggregateByKey(List[String]())(seqOp, combOp)

seqOp: (List[String], String) => List[String]
combOp: (List[String], List[String]) => List[String]
zeroVal: Unit = ()
followers: org.apache.spark.rdd.RDD[(String, List[String])] = ShuffledRDD[7] at aggregateByKey at <console>:49


In [15]:
followers.collect()

res3: Array[(String, List[String])] = Array((1,List(2)), (2,List(1)), (3,List(1, 2)))


In [19]:
val users = graphRDD.flatMap{ case (k,v) => List(k,v) }.distinct
val sourceUsers = users.subtract(following.keys)
val numVertices = 4

var ranks = followers.mapValues(v => 1.0 / numVertices)
var iterations = 10

for (i <- 1 to iterations) {
  var contribs = followers.join(ranks).values.flatMap { case (followees, rank) =>
    val size = following.size
    followees.map(followee => (followee, rank / size))
  }

  val danglingContribs = (1 - contribs.values.reduce(_ + _)) / numVertices
  contribs = contribs.reduceByKey(_ + _).mapValues(x => 0.15 / numVertices + 0.85 * (x + danglingContribs))

  val sourceContribs = sourceUsers.map(x => (x, 0.15 / numVertices + 0.85 * (0 + danglingContribs)))

  ranks = contribs.union(sourceContribs)
}

val output = ranks.collect()

<console>: 51: error: value size is not a member of org.apache.spark.rdd.RDD[(String, List[String])]